In [3]:
!pip install mediapipe --user

  Using cached mediapipe-0.10.14-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached jax-0.4.28-py3-none-any.whl.metadata (23 kB)
  Using cached jaxlib-0.4.28-cp311-cp311-win_amd64.whl.metadata (1.8 kB)
  Using cached matplotlib-3.9.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached opencv_contrib_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.4.6-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached cffi-1.16.0-cp311-cp311-win_amd64.whl.metadata (1.5 kB)
  Using cached ml_dtypes-0.4.0-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached contourpy-1.2.1-cp311-cp311-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.51.0-cp311-cp311-win_amd64.whl.metadata (162 kB)
  Using cached kiwisolver-1.4.5-cp311-cp311-win_amd64.whl.metadata 

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\DELL\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [2]:
import os # imports the 'os' module, which provides a way to interact with the operating system
import pickle #imports the 'pickle' module, which is used for serializing and deserializing Python objects
'''imports the 'Path' class from the pathlib module. 'pathlib' provides an object-oriented approach 
to working with filesystem paths. The 'Path' class represents a filesystem 'path' and offers various 
methods for working with paths and files'''
from pathlib import Path 
import mediapipe as mp #imports the mediapipe library, which is a popular framework for building various computer vision applications
import cv2 #imports the 'cv2' module, which is the Python interface for 'OpenCV'
import matplotlib.pyplot as plt #imports the 'pyplot' module from the 'matplotlib' library, which is a plotting library for Python.
import numpy as np # imports the 'numpy' library, which is a fundamental package for scientific computing with Python

'''
These lines of code import and initialize objects and modules from the 'mediapipe' library. 
These objects and modules are used for hand tracking and drawing visualizations on images or 
videos using the 'mediapipe' framework
'''
mp_hands = mp.solutions.hands # creates an instance of the 'mp_hands' object using the 'mp.solutions.hands' module from the mediapipe library

'''
1. The 'Hands' class is part of the hand tracking solution provided by the 'mediapipe' library. 
By creating an instance of this class, we can use it to perform hand tracking on images or videos.
2. 'static_image_mode=True': This parameter specifies whether the hand tracking should be optimized 
for processing static images rather than video frames.
3. 'min_detection_confidence=0.3': This parameter sets the minimum confidence value required for a 
hand detection to be considered valid. The confidence value represents the certainty of the hand detection. 
Here, it is set to 0.3, which means that a hand detection with a confidence score below 0.3 will be discarded
'''
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

DATA_DIR = './data4' #path to which the images are stored

data = [] #intended to store the data or input samples
labels = [] # intended to store the corresponding labels or categories associated with the data samples

for dir_ in os.listdir(DATA_DIR): #iterates over each image and the values are stored in 'dir_'
    '''
    This line initiates a loop that iterates over the image files in the directory specified by 'os.path.join(DATA_DIR, dir_)'. 
    The 'os.path.join()' function is used to join the 'DATA_DIR' path and 'dir_' (a subdirectory or folder name) 
    to create the complete path to the directory containing the images.
    '''
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        #These lists are used to store temporary data or variables during the processing of each image
        data_aux = [] 
        x_ = []
        y_ = []

        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path)) #reads the image file using 'cv2.imread()', which reads the image from the specified file path
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #used to convert the image from the default BGR color format to RGB format

        '''
        This processes the 'img_rgb' image using the hands object, which represents the hand tracking solution created earlier. 
        It detects and tracks hands in the image and stores the results in the 'results' variable
        '''
        results = hands.process(img_rgb)
       
        #This line checks if the multi_hand_landmarks attribute of the results variable is not None, indicating that at least one hand has been detected and tracked in the image
        if not (results.multi_hand_landmarks is None): 
                n = len(results.multi_hand_landmarks) 
                '''
                These lines check if exactly one hand is detected in the image by checking the length of the multi_hand_landmarks list. 
                If there is only one hand, the code proceeds to process the landmarks.
                '''
                if n == 1:
                    try:
                        '''
                        These lines iterate over each hand detected and tracked in the image and extract the 'x' and 'y' coordinates of 
                        each landmark (hand keypoint) using 'hand_landmarks.landmark[i].x' and 'hand_landmarks.landmark[i].y'. 
                        The x and y coordinates are appended to the 'x_' and 'y_' lists, respectively
                        '''
                        for hand_landmarks in results.multi_hand_landmarks:
                             for i in range(len(hand_landmarks.landmark)):
                                  x= hand_landmarks.landmark[i].x
                                  y= hand_landmarks.landmark[i].y
                                  x_.append(x)
                                  y_.append(y)
                             '''
                             These lines iterate over each landmark of the hand and calculate the relative 'x' and 'y' coordinates by subtracting 
                             the minimum x and y values from each coordinate. The relative coordinates are then appended to the 'data_aux' list
                             '''
                             for i in range(len(hand_landmarks.landmark)):
                                   x = hand_landmarks.landmark[i].x
                                   y = hand_landmarks.landmark[i].y
                                   data_aux.append(x - min(x_))
                                   data_aux.append(y - min(y_))
                        data.append(data_aux) #list containing the relative coordinates for the hand landmarks is appended to the 'data' list, representing the processed data for the current image
                        labels.append(dir_) #representing the label or category associated with the current image, is appended to the 'labels' list


                    except:
                            '''
                            By assigning this zero-filled array to data_aux, it provides a default value or placeholder when an exception occurs, 
                            ensuring that data_aux has a valid value even in error scenarios
                            '''
                            data_aux(np.zeros([1,63], dtype=int)[0])

'''
the code opens the file "ASL.pickle", serializes and writes the data and labels lists as a dictionary 
to the file using the pickle.dump() function, and finally closes the file. This allows you to save the 
processed data and labels in a serialized format for later use or further analysis
''' 
f = open('ASL2.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()


ModuleNotFoundError: No module named 'mediapipe'